The resultant of polynomials $f, g \in \mathbb{C}[x]$ is a polynomial in the coefficients of $f$ and $g$ that vanishes if and only if they have a common root. The resultant of polynomials $f(x) = a\prod_{i=1}^m (x - \alpha i)$ and $g(x) = b\prod_{i=1}^n(x - \beta i)$ is given by the formula
\begin{equation}
    \mathop{Res}(f, g) = a^nb^m \prod_{i=1}^m \prod_{j=1}^n (\alpha i − \beta j) = a^n \prod_{i=1}^m g(\alpha i).
\end{equation}


In [91]:
import numpy as np

def poly_strip_zeros(coeffs):
    '''
    Removes leading zero coefficients from a list.
    '''
    if all(c == 0 for c in coeffs):
        return [0]
    first_nonzero = next(i for i, c in enumerate(coeffs) if c != 0)
    return list(coeffs[first_nonzero:])

class Polynomial:
    '''
    A class to represent a polynomial and its properties.
    The polynomial is defined by its coefficients in descending order of power.
    Example: P(x) = 3x^2 - 2x + 5 is represented by [3, -2, 5].
    '''

    def __init__(self, coeffs):
        '''
        Initializes a Polynomial object.
        Args:
            coeffs: A list or numpy array of the coefficients, ordered
            from the highest power to the constant term.
        '''
        # Ensure coeffs is a list of Python ints and strip leading zeros
        cleaned_coeffs = poly_strip_zeros([int(c) for c in coeffs])
        # Use dtype=object to allow for arbitrarily large integers
        self.coeffs = np.array(cleaned_coeffs, dtype=object)

    @property
    def degree(self):
        '''
        Returns the degree of the polynomial.
        '''
        return len(self.coeffs) - 1

    @property
    def leading_coefficient(self):
        '''
        Returns the leading coefficient of the polynomial.
        '''
        return self.coeffs[0]

    def is_zero(self):
        '''
        Checks if the polynomial is the zero polynomial.
        '''
        return self.degree == -1

    def roots(self):
        '''
        Computes the roots of the polynomial.
        '''
        return np.roots(self.coeffs)

    def evaluate(self, x):
        '''
        Evaluates the polynomial at a given point or points.
        Args:
            x: A single number or a numpy array of numbers.
        Returns:
            The result of the evaluation(s).
        '''
        return np.polyval(self.coeffs, x)

    def __str__(self):
        '''
        Provides a user-friendly string representation of the polynomial.
        '''
        if self.is_zero():
            return "0"
        parts = []
        for i, coeff in enumerate(self.coeffs):
            power = self.degree - i
            if coeff == 0: continue

            coeff_str = str(coeff)
            if coeff == 1 and power != 0: coeff_str = ""
            if coeff == -1 and power != 0: coeff_str = "-"

            if power == 1: var_str = "x"
            elif power > 1: var_str = f"x^{power}"
            else: var_str = ""

            parts.append(f"{coeff_str}{var_str}")

        return " + ".join(parts).replace(" + -", " - ")

In [92]:
def compute_resultant(f, g, decimals=10):
    '''
    Computes the resultant of two Polynomial objects f and g.
    Args:
        f: A Polynomial object.
        g: A Polynomial object.
    Returns:
        The resultant of f(x) and g(x) as a complex or real number.
    '''
    if not isinstance(f, Polynomial) or not isinstance(g, Polynomial):
        raise TypeError("Inputs must be Polynomial objects.")

    if f.degree < 1 or g.degree < 1:
        raise ValueError("Input polynomials must be of degree 1 or higher.")

    # Get the necessary properties from the polynomial objects
    a = f.leading_coefficient
    n = g.degree
    alpha_roots = f.roots()

    # Evaluate g at the roots of f
    g_at_alpha_roots = g.evaluate(alpha_roots)

    # Calculate the product of the evaluations
    product_of_evaluations = np.prod(g_at_alpha_roots)

    # Apply the final formula and round to a given precision
    resultant = (a**n) * product_of_evaluations
    resultant = np.round(resultant, decimals=decimals)

    # Clean up floating point inaccuracies for real results
    if np.isclose(resultant.imag, 0):
        return resultant.real
    return resultant

f1 = Polynomial([1, 0, -4])
g1 = Polynomial([1, -5, 6])
resultant_1 = compute_resultant(f1, g1)
print(f"f(x) = {f1}")
print(f"g(x) = {g1}")
print(f"Resultant: {resultant_1}")

f2 = Polynomial([1, -2])
g2 = Polynomial([1, -3])
resultant_2 = compute_resultant(f2, g2)
print(f"\nf(x) = {f2}")
print(f"g(x) = {g2}")
print(f"Resultant: {resultant_2}")

f3 = Polynomial([1, 0, 1])
g3 = Polynomial([1, 0, 0, -1])
resultant_3 = compute_resultant(f3, g3)
print(f"\nf(x) = {f3}")
print(f"g(x) = {g3}")
print(f"Resultant: {resultant_3}")

f(x) = x^2 - 4
g(x) = x^2 - 5x + 6
Resultant: 0.0

f(x) = x - 2
g(x) = x - 3
Resultant: -1.0

f(x) = x^2 + 1
g(x) = x^3 - 1
Resultant: 2.0


The Sylvester matrix of $f(x) = a_mx^m + a_{m-1}x^{m-1}+ \cdots + a_0$ and $g(x) = b_nx^n + b_{n-1}x^{n-1}+ \cdots + b_0$ is the $(m + n) \times (m + n)$ matrix
\begin{equation}\begin{pmatrix}
    a_m   & a_{m-1}& \cdots & a_1    & a_0    & 0      & \cdots& 0      \\
    0     & a_m    & a_{m-1}& \cdots & a_1    & a_0    & \ddots& \vdots \\
    \vdots& \ddots & \ddots & \ddots & \ddots & \ddots & \ddots& 0      \\
    0     & \cdots & 0      & a_m    & a_{m-1}& \cdots & a_1   & a_0    \\
    b_n   & b_{n-1}& \cdots & b_1    & b_0    & 0      & \cdots& 0      \\
    0     & b_n    & b_{n-1}& \cdots & b_1    & b_0    & \ddots& \vdots \\
    \vdots& \ddots & \ddots & \ddots & \ddots & \ddots & \ddots& 0      \\
    0     & \cdots & 0      & b_n    & b_{n-1}& \cdots & b_1   & b_0
\end{pmatrix}\end{equation}
where the coefficients of $f$ are repeated on $n$ rows, and the coefficients of $g$ are repeated on $m$ rows.

It is known that $f$ and $g$ have a common root if and only if the Sylvester matrix is singular. This offers a robust way of computing the resultant without floating point arithmetic errors and use of root finding libraries.

For the forward implication, assume that $f$ and $g$ have a common root $\alpha$ so that $f(\alpha) = g(\alpha) = 0$. Then we can write
\begin{align}
    f(x) &= (x - \alpha) f_1(x), \quad \deg(f_1) = m - 1; \\
    g(x) &= (x - \alpha) g_1(x), \quad \deg(g_1) = n - 1.
\end{align}
Cross multiplying, we obtain
\begin{align}
    g_1(x)f(x) = g_1(x)(x - \alpha)f_1(x), \\
    f_1(x)g(x) = f_1(x)(x - \alpha)g_1(x).
\end{align}
This leads to the identity $g_1(x)f(x) - f_1(x)g(x) = 0$. This equation can be expressed as a system of linear equations in terms of the coefficients of $g_1(x)$ and $-f_1(x)$. This system is represented by the transpose of the Sylvester matrix. Because a non-trivial solution for the coefficients of $g_1(x)$ and $-f_1(x)$ exists, the matrix of the system must be singular, so the determinant of the transpose of the Sylvester matrix, (hence the matrix itself), must be zero.

Conversely, let us assume that the Sylvester matrix is singular so its determinant is zero. This implies that there is a non-zero vector in its kernel which corresponds to the existence of two non-zero polynomials $p(x)$ of degree at most $n-1$ and $q(x)$ of degree at most $m-1$, such that $p(x)f(x) + q(x)g(x) = 0$, which we rewrite as $p(x)f(x) = -q(x)g(x)$. The set of polynomials over a field is a UFD, which means that $f(x)$ must divide the product $-q(x)g(x)$.

Assume that $f$ and $g$ are coprime, so $f(x)$ must divide $q(x)$. Since $\deg(q) < \deg(f)$, this forces $q(x)$ to be the zero polynomial, which means that $p(x)f(x) = 0$. Since $f(x)$ is not zero, $p(x)$ must also be the zero polynomial. This contradicts the fact that $p(x)$ and $q(x)$ were non-zero polynomials. Thus, $f$ and $g$ must have a common root.

In [93]:
def compute_sylvester_determinant(f, g):
    '''
    Constructs the Sylvester matrix for two Polynomial objects and
    computes its determinant.
    Args:
        f: A Polynomial object.
        g: A Polynomial object.

    Returns:
        The determinant of the Sylvester matrix.
    '''
    if not isinstance(f, Polynomial) or not isinstance(g, Polynomial):
        raise TypeError("Inputs must be Polynomial objects.")

    m = f.degree
    n = g.degree
    f_coeffs = f.coeffs
    g_coeffs = g.coeffs

    # The dimension of the Sylvester matrix is (m + n) x (m + n)
    dim = m + n
    sylvester_matrix = np.zeros((dim, dim), dtype=complex)

    # Populate the first n rows with coefficients of f(x)
    for i in range(n):
        sylvester_matrix[i, i:i + m + 1] = f_coeffs

    # Populate the next m rows with coefficients of g(x)
    for i in range(m):
        sylvester_matrix[i + n, i:i + n + 1] = g_coeffs

    # Compute and return the determinant
    determinant = np.linalg.det(sylvester_matrix)

    # Clean up floating point inaccuracies for real results
    if np.isclose(determinant.imag, 0):
        return determinant.real
    return determinant

f1 = Polynomial([1, 0, -4])
g1 = Polynomial([1, -5, 6])
det1 = compute_sylvester_determinant(f1, g1)
print(f"f(x) = {f1}")
print(f"g(x) = {g1}")
print(f"Sylvester determinant: {det1}")

f2 = Polynomial([1, -2])
g2 = Polynomial([1, -3])
det2 = compute_sylvester_determinant(f2, g2)
print(f"\nf(x) = {f2}")
print(f"g(x) = {g2}")
print(f"Sylvester determinant: {det2}")

f3 = Polynomial([1, 0, 1])
g3 = Polynomial([1, 0, 0, -1])
det3 = compute_sylvester_determinant(f3, g3)
print(f"\nf(x) = {f3}")
print(f"g(x) = {g3}")
print(f"Sylvester determinant: {det3}")

f(x) = x^2 - 4
g(x) = x^2 - 5x + 6
Sylvester determinant: 0.0

f(x) = x - 2
g(x) = x - 3
Sylvester determinant: -1.0

f(x) = x^2 + 1
g(x) = x^3 - 1
Sylvester determinant: 2.0


The resultant has the following properties.
\begin{align}
    \mathop{Res}(f, g) &= (−1)^{\deg(f)\deg(g)} \mathop{Res}(g, f) &&\text{ for } f, g \in \mathbb{C}[x], \\
    \mathop{Res}(\lambda f, \mu g) &= \lambda^{\deg(g)}\mu^{\deg(f)} \mathop{Res}(f, g) &&\text{ for } f, g \in \mathbb{C}[x] \text{ and } \lambda, \mu \in \mathbb{C}, \\
    \mathop{Res}(f, gh) &= \mathop{Res}(f, g) \mathop{Res}(f, h) &&\text{ for } f, g, h \in \mathbb{C}[x], \\
    \mathop{Res}(f, g) &= \mathop{Res}(f, g + hf) &&\text{ for } f, g, h \in \mathbb{C}[x] \text{ with $f$ monic}.
\end{align}

The following function implements polynomial pseudo-division. It takes two polynomials $f$ and $g$ with integer coefficients, then finds an integer $c$ and polynomials $q$ and $r$ such that $cf = qg + r$ with $\deg(r) < \deg(g)$. This will allow us to compute the resultant only using integer arithmetic.

In [94]:
def pseudo_division(f, g):
    '''
    Performs pseudo-division on two Polynomial objects.
    Args:
        f: The dividend Polynomial.
        g: The divisor Polynomial.
    Returns:
        A tuple (c, q, r) where c is an integer, q and r are Polynomials,
        and c*f = q*g + r.
    '''
    m, n = f.degree, g.degree
    if m < n:
        return 1, Polynomial([0]), f

    b_n = g.leading_coefficient
    r = Polynomial(f.coeffs)
    q = Polynomial([0])
    c = 1

    delta = m - n

    # Implementation of the pseudo-division or polynomial remainder sequence) algorithm
    r_coeffs = list(r.coeffs)

    for i in range(delta + 1):
        q_term_coeff = r_coeffs[0]
        q_term_deg = m - n - i

        # Multiply remainder by b_n
        r_coeffs = [b_n * coeff for coeff in r_coeffs]

        # Calculate next term of quotient
        g_mult = [q_term_coeff * coeff for coeff in g.coeffs]

        # Subtract from remainder
        for j in range(len(g.coeffs)):
            r_coeffs[j] -= g_mult[j]

        r_coeffs.pop(0)

    c = b_n ** (delta + 1)
    # The quotient is not needed for the resultant calculation, but we return it.
    # We return a placeholder for the quotient but the remainder is the key part.
    r = Polynomial(r_coeffs if r_coeffs else [0])

    return c, Polynomial([0]), r

Our algorithm computes $\mathop{Res}(f, g)$ by repeatedly reducing the degree of the polynomials involved:
*   Base case: If $g$ is a constant, then the recursion stops. The resultant is simply the constant $g$ raised to the power of the degree of $f$.
*   Swapping: The algorithm requires $\deg(f) \geq \deg(g)$. If not, it swaps them, which introduces a sign change of $(-1)^{\deg(f)\deg(g)}$.
*   Recursive Step: Uses pseudo division to find a remainder $r$ such that $\deg(r) < \deg(g)$. We use the recursive formula
\begin{equation}
    \mathop{Res}(f, g) = (-1)^{mn} \frac{b_n^{m-k}}{c^n} \mathop{Res}(g, r),
\end{equation}
where $b_n$ is the leading coefficient of $g$, $k = \deg(r)$ and $c$ is the multiplier from pseudo division. The algorithm calls itself with $(g, r)$. If the remainder $r$ is zero, then $f$ and $g$ have a common factor, so the resultant is $0$.
*   This process guarantees termination because the degree of the second polynomial decreases with each recursive call.

In [95]:
def resultant_integer(f, g):
    '''
    Computes the resultant of two Polynomial objects in Z[x] using only
    integer arithmetic.
    '''
    if not isinstance(f, Polynomial) or not isinstance(g, Polynomial):
        raise TypeError("Inputs must be Polynomial objects.")

    m, n = f.degree, g.degree

    # Base Case: if g is a constant b0, Res(f, g) = b0^m
    if n == 0:
        return g.leading_coefficient ** m

    # If f is the zero polynomial, resultant is 0
    if f.is_zero():
        return 0

    # Recursive Step
    if m < n:
        # Swap polynomials: Res(f,g) = (-1)^(mn) * Res(g,f)
        sign = -1 if (m * n) % 2 != 0 else 1
        return sign * resultant_integer(g, f)
    else:
        # Perform pseudo-division: c*f = q*g + r
        c, _, r = pseudo_division(f, g)

        # If remainder is zero, polynomials have a common root
        if r.is_zero():
            return 0

        k = r.degree
        b_n = g.leading_coefficient

        # Apply the recursive formula
        sign = -1 if (m * n) % 2 != 0 else 1

        recursive_res = resultant_integer(g, r)

        # This division is guaranteed to be exact
        num = sign * (b_n ** (m - k)) * recursive_res
        den = c

        return num // den

f1 = Polynomial([1, 0, -4])
g1 = Polynomial([1, -5, 6])
print(f"Res({f1}, {g1}) = {resultant_integer(f1, g1)}")

f2 = Polynomial([1, -2])
g2 = Polynomial([1, -3])
print(f"Res({f2}, {g2}) = {resultant_integer(f2, g2)}")

f3 = Polynomial([1, 0, 1])
g3 = Polynomial([1, 0, 0, -1])
print(f"Res({f3}, {g3}) = {resultant_integer(f3, g3)}")

Res(x^2 - 4, x^2 - 5x + 6) = 0
Res(x - 2, x - 3) = -1
Res(x^2 + 1, x^3 - 1) = 2


Reducing the coefficients of polynomials $f$ and $g$ modulo prime $p$, the polynomials $f_p(x)$ and $g_p(x)$ are not coprime in the field $\mathbb{F}_p[x]$ if and only if their resultant Res(f_p, g_p) is zero in $\mathbb{F}_p$.

The resultant is calculated through arithmetic operations on the polynomials' coefficients. The operation of reducing coefficients modulo $p$ commutes with these arithmetic operations. Therefore, the resultant of the modularly reduced polynomials is the modularly reduced resultant of the original polynomials
\begin{equation}
\mathop{Res}(f_p, g_p) = \mathop{Res}(f, g) \mod{p}.
\end{equation}
provided the degrees of $f$ and $g$ do not both decrease when reduced modulo $p$. A prime $p$ is exceptional if $f_p(x)$ and $g_p(x)$ are not coprime, or equivalently, the exceptional primes are the prime divisors of the integer resultant $\mathop{Res}(f, g)$. There are only finitely many such exceptional primes since the resultant is finite.

In [96]:
def get_prime_factors(n):
    '''
    Finds the prime factors of an integer n.
    '''
    factors = set()
    n = abs(n)
    d = 2
    while d * d <= n:
        while n % d == 0:
            factors.add(d)
            n //= d
        d += 1
    if n > 1: factors.add(n)
    return sorted(list(factors))

f1 = Polynomial([1, -3, 2, 1])
g1 = Polynomial([2, -1, 1])
res1 = resultant_integer(f1, g1)
factors1 = get_prime_factors(res1)
print(f"Res({f1}, {g1}) = {resultant_integer(f1, g1)}")
print(f"Exceptional primes: {factors1}")

f2 = Polynomial([1, 4, 5, 13])
g2 = Polynomial([3, 2, 4, -9])
res2 = resultant_integer(f2, g2)
factors2 = get_prime_factors(res2)
print(f"\nRes({f2}, {g2}) = {resultant_integer(f2, g2)}")
print(f"Exceptional primes: {factors2}")

f3 = Polynomial([1, 0, 0, 1, -9, 25])
g3 = Polynomial([2, 7, 31, 69])
res3 = resultant_integer(f3, g3)
factors3 = get_prime_factors(res3)
print(f"\nRes({f3}, {g3}) = {resultant_integer(f3, g3)}")
print(f"Exceptional primes: {factors3}")

f4 = Polynomial([1, 0, 0, 0, 7, 1, -3])
g4 = Polynomial([1, 0, 0, 3, 31, 10])
res4 = resultant_integer(f4, g4)
factors4 = get_prime_factors(res4)
print(f"\nRes({f4}, {g4}) = {resultant_integer(f4, g4)}")
print(f"Exceptional primes: {factors4}")

Res(x^3 - 3x^2 + 2x + 1, 2x^2 - x + 1) = 172
Exceptional primes: [2, 43]

Res(x^3 + 4x^2 + 5x + 13, 3x^3 + 2x^2 + 4x - 9) = -36659700
Exceptional primes: [2, 3, 5, 7, 11, 23]

Res(x^5 + x^2 - 9x + 25, 2x^3 + 7x^2 + 31x + 69) = 3195445644660928
Exceptional primes: [2, 257, 751, 787, 1279]

Res(x^6 + 7x^2 + x - 3, x^5 + 3x^2 + 31x + 10) = -71994888005495575428
Exceptional primes: [2, 3, 7, 11, 96737]
